In [3]:
# !pip install datasets transformers pandasql beautifultable -qqq

In [5]:
prefix = "translate english to sparql2: "
model_name = "yazdipour/text-to-sparql-t5-small-qald9"

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, T5TokenizerFast
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")
tokenizer = T5TokenizerFast.from_pretrained(model_name)

In [8]:
translator = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=0 #0 for cuda, -1 for cpu
)

In [9]:
def replace_all(text, dict):
    for i, j in dict.items():
        text = text.replace(i, j)
    return text

In [2]:
def decode_props(qry):
    return qry.replace("[", "{").replace("]", "}")

In [47]:
def translate(q):
    tr = translator(prefix+q, max_length=100)[0]['translation_text']
    return tr

In [48]:
from beautifultable import BeautifulTable
def pretty_translate(t,q): 
    ans = translate(q)
    table = BeautifulTable(maxwidth=140)
    table.rows.append(['QUESTION', q])
    table.rows.append(['Target', t])
    table.rows.append(['RESULT', ans])
    print(table)

# TESTING

In [49]:
prefix

'translate english to sparql2: '

In [52]:
translate('How old is Bill Gates?')

'select distinct ?uri where [ ?uri dbr:bill_gates dbo:date ?uri ]'

In [53]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+----------+--------------------------------------------------------------------------------+
| QUESTION |             Who is the country for head of state of Mahmoud Abbas?             |
+----------+--------------------------------------------------------------------------------+
|  Target  | select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 } |
+----------+--------------------------------------------------------------------------------+
|  RESULT  |  select distinct ?uri where [ ?uri dbr:Mahmoud_Abbas dbo:head_of_state ?uri ]  |
+----------+--------------------------------------------------------------------------------+


In [54]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }',"What country is Mahmoud Abbas the head of state of?")

+----------+--------------------------------------------------------------------------------+
| QUESTION |              What country is Mahmoud Abbas the head of state of?               |
+----------+--------------------------------------------------------------------------------+
|  Target  | select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 } |
+----------+--------------------------------------------------------------------------------+
|  RESULT  |  select distinct ?uri where [ ?uri dbr:Mahmoud_Abbas dbo:head_of_state ?uri ]  |
+----------+--------------------------------------------------------------------------------+


In [55]:
pretty_translate('select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }','Who is the country for head of state of Mahmoud Abbas?')

+----------+--------------------------------------------------------------------------------+
| QUESTION |             Who is the country for head of state of Mahmoud Abbas?             |
+----------+--------------------------------------------------------------------------------+
|  Target  | select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 } |
+----------+--------------------------------------------------------------------------------+
|  RESULT  |  select distinct ?uri where [ ?uri dbr:Mahmoud_Abbas dbo:head_of_state ?uri ]  |
+----------+--------------------------------------------------------------------------------+


In [56]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress is the voice over on South Park and is employed as a singer?".lower())

+----------+-----------------------------------------------------------------------------------------------------------+
| QUESTION |             which female actress is the voice over on south park and is employed as a singer?             |
+----------+-----------------------------------------------------------------------------------------------------------+
|  Target  |             SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}              |
+----------+-----------------------------------------------------------------------------------------------------------+
|  RESULT  | select distinct ?sbj where [ ?sbj wdt:voice_over wd:south_park . ?sbj wdt:instance_of wd:female_actress ] |
+----------+-----------------------------------------------------------------------------------------------------------+


In [57]:
pretty_translate("SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220}","Which female actress on South Park is the voice over and is used as a singer?")

+----------+----------------------------------------------------------------------------------+
| QUESTION |  Which female actress on South Park is the voice over and is used as a singer?   |
+----------+----------------------------------------------------------------------------------+
|  Target  | SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?answer . ?answer wdt:P106 wd:Q177220} |
+----------+----------------------------------------------------------------------------------+
|  RESULT  |      select distinct ?uri where [ ?uri dbr:South_Park dbo:voice_over ?uri ]      |
+----------+----------------------------------------------------------------------------------+


In [58]:
# Paul Erdős (Q173746)
pretty_translate("select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}","Which is the PIM authority ID of Paul Erd?")

+----------+------------------------------------------------------------------------+
| QUESTION |               Which is the PIM authority ID of Paul Erd?               |
+----------+------------------------------------------------------------------------+
|  Target  |     select distinct ?answer where { wd:Q173746 wdt:P3973 ?answer}      |
+----------+------------------------------------------------------------------------+
|  RESULT  | select distinct ?uri where [ ?uri dbr:Paul_Erd dbo:authority_ID ?uri ] |
+----------+------------------------------------------------------------------------+


In [59]:
pretty_translate("SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }", 
            "What was the population of Somalia in 2009-0-0?")

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                        What was the population of Somalia in 2009-0-0?                                        |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  |        SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),'2009')) }         |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  | select ?obj where [ dbr:Somala dbo:population ?obj . ?obj dbo:population ?obj . ?obj dbo:population ?obj . ?obj dbo:populatio |
|          |                                                           n ?obj ]                                                            |
+----------+-

In [60]:
translate('Humans born in New York City') #random query - answer seems correct

'select distinct ?uri where [ dbr:human dbo:place_of_birth dbo:New_York_City ]'

In [61]:
# From QALD
target = "ASK WHERE { <http://dbpedia.org/resource/Taiko> a <http://dbpedia.org/class/yago/WikicatJapaneseMusicalInstruments> }"
q =  "Are Taiko some kind of Japanese musical instrument?"
pretty_translate(target, q)

+----------+-----------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                  Are Taiko some kind of Japanese musical instrument?                                  |
+----------+-----------------------------------------------------------------------------------------------------------------------+
|  Target  | ASK WHERE { <http://dbpedia.org/resource/Taiko> a <http://dbpedia.org/class/yago/WikicatJapaneseMusicalInstruments> } |
+----------+-----------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                                 ask where [ dbr:Taiko dbo:instrument dbo:type ?obj ]                                  |
+----------+-----------------------------------------------------------------------------------------------------------------------+


In [62]:
# From QALD
target = "PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?uri dct:subject dbc:Assassins_of_Julius_Caesar }"
q =  "Who killed Caesar?"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                                      Who killed Caesar?                                                       |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT DISTINCT ?uri WHERE { ?uri |
|          |                                          dct:subject dbc:Assassins_of_Julius_Caesar }                                         |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                                 select distinct ?uri where [ ?uri dbr:Casin dbo:killer ?uri ]                                 |
+----------+-

In [63]:
# From QALD
q = 'What is the highest mountain in Germany?'
target = "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: <http://dbpedia.org/ontology/>  \nSELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation ?elevation ; onto:locatedInArea <http://dbpedia.org/resource/Germany> } ORDER BY DESC(?elevation) LIMIT 1"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                           What is the highest mountain in Germany?                                            |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: < |
|          |                                                http://dbpedia.org/ontology/>                                                  |
|          | SELECT ?uri WHERE { ?uri rdf:type onto:Mountain ; onto:elevation ?elevation ; onto:locatedInArea <http://dbpedia.org/resource |
|          |                                         /Germany> } ORDER BY DESC(?elevation) LIMIT 1                                         |
+----------+-

In [64]:
# From QALD
q = 'Which American presidents were in office during the Vietnam War?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT ?uri WHERE { ?uri dct:subject dbc:Presidents_of_the_United_States . res:Vietnam_War dbo:commander ?uri }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                               Which American presidents were in office during the Vietnam War?                                |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX dct: <http://purl.org/dc/terms/> |
|          |  PREFIX dbc: <http://dbpedia.org/resource/Category:> SELECT ?uri WHERE { ?uri dct:subject dbc:Presidents_of_the_United_States |
|          |                                             . res:Vietnam_War dbo:commander ?uri }                                            |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  | 

In [65]:
# From QALD
q = 'How many gold medals did Michael Phelps win at the 2008 Olympics?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT Count(?sub) as ?c WHERE { ?sub dbo:goldMedalist dbr:Michael_Phelps . filter (contains (str(?sub), \"2008\") && contains (str(?sub), \"Olympics\")) }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                               How many gold medals did Michael Phelps win at the 2008 Olympics?                               |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/ |
|          | 22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT Count(?sub) as ?c WHERE { ?sub dbo:goldMedalis |
|          |              t dbr:Michael_Phelps . filter (contains (str(?sub), "2008") && contains (str(?sub), "Olympics")) }               |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  | 

In [66]:
# From QALD
q = 'What is the profession of Frank Herbert?'
target = "PREFIX dbpedia2: <http://dbpedia.org/property/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?string WHERE { res:Frank_Herbert dbpedia2:occupation ?string }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                           What is the profession of Frank Herbert?                                            |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX dbpedia2: <http://dbpedia.org/property/> PREFIX res: <http://dbpedia.org/resource/> SELECT DISTINCT ?string WHERE { re |
|          |                                         s:Frank_Herbert dbpedia2:occupation ?string }                                         |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                           select distinct ?uri where [ ?uri dbr:Frank_Horror dbo:occupation ?uri ]                            |
+----------+-

In [67]:
# From QALD
q = 'How many seats does the home stadium of FC Porto have?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> PREFIX dbr: <http://dbpedia.org/resource/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX db: <http://dbpedia.org/> SELECT ?capacity WHERE { { dbr:FC_Porto dbo:ground ?ground . ?ground dbo:capacity ?capacity } UNION { dbr:FC_Porto dbo:ground ?ground . ?ground dbp:capacity ?capacity } }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                    How many seats does the home stadium of FC Porto have?                                     |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbp: <http://dbpedia.org/property/> PREFIX dbr: <http://dbpedia.org/resourc |
|          | e/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX db: |
|          |  <http://dbpedia.org/> SELECT ?capacity WHERE { { dbr:FC_Porto dbo:ground ?ground . ?ground dbo:capacity ?capacity } UNION {  |
|          |                             dbr:FC_Porto dbo:ground ?ground . ?ground dbp:capacity ?capacity } }                              |
+----------+-

In [68]:
# From QALD
q = 'Which frequent flyer program has the most airlines?'
target = "SELECT ?uri WHERE { ?airline <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Airline> . ?airline <http://dbpedia.org/property/frequentFlyer> ?uri. } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?airline)) OFFSET 0 LIMIT 1"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                      Which frequent flyer program has the most airlines?                                      |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT ?uri WHERE { ?airline <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Airline> . ?airli |
|          | ne <http://dbpedia.org/property/frequentFlyer> ?uri. } GROUP BY ?uri ORDER BY DESC(COUNT(DISTINCT ?airline)) OFFSET 0 LIMIT 1 |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |         select distinct ?uri where [ ?uri dbr:Frequent_Flyer_Programm dbo:Airlines ?uri ] order BY DESC(?uri )limit 5         |
+----------+-

In [69]:
# From QALD
q = 'Which European countries have a constitutional monarchy?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/Category:> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?uri WHERE { ?uri dct:subject dbc:Countries_in_Europe ; dbo:governmentType dbr:Constitutional_monarchy }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                   Which European countries have a constitutional monarchy?                                    |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dct: <http://purl.org/dc/terms/> PREFIX dbc: <http://dbpedia.org/resource/C |
|          | ategory:> PREFIX dbr: <http://dbpedia.org/resource/> SELECT ?uri WHERE { ?uri dct:subject dbc:Countries_in_Europe ; dbo:gover |
|          |                                            nmentType dbr:Constitutional_monarchy }                                            |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  | 

In [70]:
# From QALD
q = 'Which countries have places with more than two caves?'
target = "PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHERE { ?cave rdf:type dbo:Cave ; dbo:location ?uri . ?uri rdf:type dbo:Country } GROUP BY ?uri HAVING ( COUNT(?cave) > 2 )"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                     Which countries have places with more than two caves?                                     |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> SELECT DISTINCT ?uri WHE |
|          |   RE { ?cave rdf:type dbo:Cave ; dbo:location ?uri . ?uri rdf:type dbo:Country } GROUP BY ?uri HAVING ( COUNT(?cave) > 2 )    |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                                select distinct ?uri where [ ?uri dbr:country dbo:place ?uri ]                                 |
+----------+-

In [71]:
# From QALD
q = 'Which airports are located in California, USA?'
target = "SELECT DISTINCT ?uri WHERE { ?uri a <http://dbpedia.org/ontology/Airport> { ?uri <http://dbpedia.org/ontology/location> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> ?city . ?city <http://dbpedia.org/ontology/isPartOf> <http://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/operator> <http://dbpedia.org/resource/California> } }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                        Which airports are located in California, USA?                                         |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?uri WHERE { ?uri a <http://dbpedia.org/ontology/Airport> { ?uri <http://dbpedia.org/ontology/location> <http |
|          | ://dbpedia.org/resource/California> } UNION { ?uri <http://dbpedia.org/ontology/city> <http://dbpedia.org/resource/California |
|          | > } UNION { ?uri <http://dbpedia.org/ontology/city> ?city . ?city <http://dbpedia.org/ontology/isPartOf> <http://dbpedia.org/ |
|          |    resource/California> } UNION { ?uri <http://dbpedia.org/ontology/operator> <http://dbpedia.org/resource/California> } }    |
+----------+-

In [72]:
# From QALD
q = "What are the nicknames of San Francisco?"
target = "SELECT DISTINCT ?string WHERE { res:San_Francisco foaf:nick ?string }"
pretty_translate(target, q)

+----------+-----------------------------------------------------------------------+
| QUESTION |               What are the nicknames of San Francisco?                |
+----------+-----------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?string WHERE { res:San_Francisco foaf:nick ?string } |
+----------+-----------------------------------------------------------------------+
|  RESULT  |   select distinct ?uri where [ ?uri dbr:San_Francio dbo:name ?uri ]   |
+----------+-----------------------------------------------------------------------+


In [73]:
# From QALD
q = "What is Angela Merkel’s birth name?"
target = "SELECT DISTINCT ?string WHERE { res:Angela_Merkel dbp:birthName ?string }"
pretty_translate(target, q)

+----------+---------------------------------------------------------------------------+
| QUESTION |                    What is Angela Merkel’s birth name?                    |
+----------+---------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?string WHERE { res:Angela_Merkel dbp:birthName ?string } |
+----------+---------------------------------------------------------------------------+
|  RESULT  |    select distinct ?sbj where [ ?sbj wdt:birth_name wd:angela_franca ]    |
+----------+---------------------------------------------------------------------------+


In [74]:
# From QALD
q = "Who is the mayor of Berlin?"
target = "SELECT DISTINCT ?uri WHERE { res:Berlin dbp:leader ?uri }"
pretty_translate(target, q)

+----------+----------------------------------------------------------------------------+
| QUESTION |                        Who is the mayor of Berlin?                         |
+----------+----------------------------------------------------------------------------+
|  Target  |         SELECT DISTINCT ?uri WHERE { res:Berlin dbp:leader ?uri }          |
+----------+----------------------------------------------------------------------------+
|  RESULT  | select distinct ?uri where [ ?uri dbr:Berlin dbo:head_of_government ?uri ] |
+----------+----------------------------------------------------------------------------+


In [75]:
# From QALD
q = "Which software has been published by Mean Hamster Software?"
target = "SELECT DISTINCT ?uri WHERE { ?uri rdf:type onto:Software { ?uri prop:publisher \"Mean Hamster Software\"@en } UNION { ?uri onto:publisher res:Mean_Hamster_Software } }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                  Which software has been published by Mean Hamster Software?                                  |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?uri WHERE { ?uri rdf:type onto:Software { ?uri prop:publisher "Mean Hamster Software"@en } UNION { ?uri onto |
|          |                                           :publisher res:Mean_Hamster_Software } }                                            |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                      select distinct ?uri where [ ?uri dbo:publication_software dbo:published_by ?uri ]                       |
+----------+-

In [76]:
# From QALD
q = "Which country was Bill Gates born in?"
target = "SELECT DISTINCT ?country WHERE { { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:country ?country } UNION { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:isPartOf ?place . ?place dbo:country ?country } }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                             Which country was Bill Gates born in?                                             |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?country WHERE { { dbr:Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:country ?country } UNION { dbr |
|          |          :Bill_Gates dbo:birthPlace ?birthPlace . ?birthPlace dbo:isPartOf ?place . ?place dbo:country ?country } }           |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                              select distinct ?uri where [ ?uri dbr:bill_gates dbo:country ?uri ]                              |
+----------+-

In [77]:
# From QALD
q = "How many grand-children did Jacques Cousteau have?"
target = "SELECT COUNT(DISTINCT ?y AS ?y) WHERE { <http://dbpedia.org/resource/Jacques_Cousteau> <http://dbpedia.org/ontology/child> ?x . ?x <http://dbpedia.org/ontology/child> ?y . }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                      How many grand-children did Jacques Cousteau have?                                       |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT COUNT(DISTINCT ?y AS ?y) WHERE { <http://dbpedia.org/resource/Jacques_Cousteau> <http://dbpedia.org/ontology/child> ?x |
|          |                                        . ?x <http://dbpedia.org/ontology/child> ?y . }                                        |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                        select distinct ?uri where [ ?uri dbr:James_Coursteau dbo:grand-children ?uri ]                        |
+----------+-

In [78]:
# From QALD
q = "Give me all professional skateboarders from Sweden."
target = "SELECT DISTINCT ?uri WHERE { ?uri dbo:occupation dbr:Skateboarder { ?uri dbo:birthPlace dbr:Sweden } UNION { ?uri dbo:birthPlace ?place . ?place dbo:country dbr:Sweden } }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                      Give me all professional skateboarders from Sweden.                                      |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?uri WHERE { ?uri dbo:occupation dbr:Skateboarder { ?uri dbo:birthPlace dbr:Sweden } UNION { ?uri dbo:birthPl |
|          |                                        ace ?place . ?place dbo:country dbr:Sweden } }                                         |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                            select distinct ?uri where [ ?uri dbr:Skiboarder dbo:profession ?uri ]                             |
+----------+-

In [79]:
# From QALD
q = "Which monarchs of the United Kingdom were married to a German?"
target = "SELECT DISTINCT ?uri WHERE { ?uri rdf:type yago:WikicatMonarchsOfTheUnitedKingdom ; dbo:spouse ?spouse . ?spouse dbo:birthPlace res:Germany }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                Which monarchs of the United Kingdom were married to a German?                                 |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?uri WHERE { ?uri rdf:type yago:WikicatMonarchsOfTheUnitedKingdom ; dbo:spouse ?spouse . ?spouse dbo:birthPla |
|          |                                                       ce res:Germany }                                                        |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                    select distinct ?uri where [ ?uri dbr:Monarchs_of_the_united_kingdom dbo:spouse ?uri ]                     |
+----------+-

In [80]:
# From QALD
q = "Give me all Argentine films."
target = "SELECT DISTINCT ?uri WHERE { { ?uri rdf:type yago:ArgentineFilms } UNION { ?uri rdf:type dbo:Film { ?uri dbo:country res:Argentina } UNION { ?uri dbp:country \"Argentina\"@en } } }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                                 Give me all Argentine films.                                                  |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?uri WHERE { { ?uri rdf:type yago:ArgentineFilms } UNION { ?uri rdf:type dbo:Film { ?uri dbo:country res:Arge |
|          |                                     ntina } UNION { ?uri dbp:country "Argentina"@en } } }                                     |
+----------+-------------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                               select distinct ?uri where [ ?uri dbr:Argentine dbo:films ?uri ]                                |
+----------+-

In [81]:
# From QALD
q = "How did Michael Jackson die?"
target = "SELECT DISTINCT ?s WHERE { <http://dbpedia.org/resource/Michael_Jackson> <http://dbpedia.org/property/deathCause> ?s }"
pretty_translate(target, q)

+----------+------------------------------------------------------------------------------------------------------------------------+
| QUESTION |                                              How did Michael Jackson die?                                              |
+----------+------------------------------------------------------------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?s WHERE { <http://dbpedia.org/resource/Michael_Jackson> <http://dbpedia.org/property/deathCause> ?s } |
+----------+------------------------------------------------------------------------------------------------------------------------+
|  RESULT  |                            select distinct ?uri where [ dbr:Michael_James dbo:death ?uri ]                             |
+----------+------------------------------------------------------------------------------------------------------------------------+


In [82]:
# From QALD
q = "Where did Michael Jackson died?"
target = "."
pretty_translate(target, q)

+----------+--------------------------------------------------------------------------+
| QUESTION |                     Where did Michael Jackson died?                      |
+----------+--------------------------------------------------------------------------+
|  Target  |                                    .                                     |
+----------+--------------------------------------------------------------------------+
|  RESULT  | select distinct ?uri where [ dbr:Michael_James dbo:place_of_death ?uri ] |
+----------+--------------------------------------------------------------------------+


In [83]:
# From QALD
q = "Which classes does the Millepede belong to?"
target = "SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String }"
pretty_translate(target, q)

+----------+-------------------------------------------------------------------+
| QUESTION |            Which classes does the Millepede belong to?            |
+----------+-------------------------------------------------------------------+
|  Target  | SELECT DISTINCT ?String WHERE { res:Millipede dbp:taxon ?String } |
+----------+-------------------------------------------------------------------+
|  RESULT  | select distinct ?uri where [ ?uri dbr:Millepede dbo:class ?uri ]  |
+----------+-------------------------------------------------------------------+
